# BERT Self-Attention

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from bertviz import head_view

model_id = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, output_attentions=True).eval()

text = "City in northern Germany that students like"
enc = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    out = model(**enc)

tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"][0])[1:-1]  # ohne CLS/SEP
atts = [L[:, :, 1:-1, 1:-1] for L in out.attentions]

# Head: Auswahl über die 12 Farben 
# → Jeder Head = eigener „Blickwinkel“ auf Sprache
# Head 0: Nachbarschaften / Positionsbezug
# Head 3: Grammatik / Satzstruktur (z. B. Determiner–Nomen)
# Head 6: Subjekt–Verb / syntaktische Rollen
# Head 9: Langdistanz-Bezüge / Relativsätze
# Head 11: Semantik / Satzfokus, Zusammenfassung

# Layer: Auswahl in Dropdown-Menü 
# → Je tiefer der Layer, desto abstrakter das Verständnis
# Layer 0: Wortformen, lokale Muster
# Layer 3: Phrasen, kurze Abhängigkeiten
# Layer 6: Satzstruktur, grammatische Beziehungen
# Layer 9: Semantische Zusammenhänge, Relativsätze
# Layer 11: Globale Bedeutung, Kontextintegration

head_view(atts, tokens=tokens, heads=[10], prettify_tokens=True)


<IPython.core.display.Javascript object>

# Bias in GPT-2

In [14]:
from bertviz import head_view
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2", output_attentions=True)

def generate_attention_visualization(text):

    inputs = tokenizer.encode(
        text, return_tensors="pt"
    )
    outputs = model(inputs)
    attention = outputs[-1]
    tokens = tokenizer.convert_ids_to_tokens(inputs[0])

    return head_view(attention, tokens, layer=5, heads=[10])

In [15]:
generate_attention_visualization("The doctor asked the nurse a question. He")

<IPython.core.display.Javascript object>

In [16]:
generate_attention_visualization("The doctor asked the nurse a question. She")

<IPython.core.display.Javascript object>